<a href="https://colab.research.google.com/github/Jogging97/S20DL/blob/master/aum_cifar10_vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/models/*.py

'/content/gdrive/My Drive/Colab Notebooks/models/densenet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/dla.py'
'/content/gdrive/My Drive/Colab Notebooks/models/dla_simple.py'
'/content/gdrive/My Drive/Colab Notebooks/models/dpn.py'
'/content/gdrive/My Drive/Colab Notebooks/models/efficientnet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/googlenet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/__init__.py'
'/content/gdrive/My Drive/Colab Notebooks/models/lenet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/mobilenet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/mobilenetv2.py'
'/content/gdrive/My Drive/Colab Notebooks/models/pnasnet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/preact_resnet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/regnet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/resnet.py'
'/content/gdrive/My Drive/Colab Notebooks/models/resnext.py'
'/content/gdrive/My Drive/Colab Notebooks/models/senet.py'
'/content/gdri

In [4]:
!cat '/content/gdrive/My Drive/Colab Notebooks/models/utils.py'

cat: '/content/gdrive/My Drive/Colab Notebooks/models/utils.py': No such file or directory


In [5]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/models')
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [6]:
!ls /content/gdrive/My\ Drive/Colab\ Notebooks/*.py

'/content/gdrive/My Drive/Colab Notebooks/utils.py'


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Mar 26 10:13:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
'''Train CIFAR10 with PyTorch.'''
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

import vgg
import utils
# from models import *
# from utils import progress_bar

!pip install aum
from aum import AUMCalculator, DatasetWithIndex
import pandas as pd
# from remove_samples import Remove_Samples


In [9]:
save_dir = '/content/gdrive/My Drive/Colab Notebooks/aum_score/vgg_'
aum_calculator = AUMCalculator(save_dir, compressed=True)

# parser = argparse.ArgumentParser(description='PyTorch CIFAR10 Training')
# parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
# parser.add_argument('--resume', '-r', action='store_true',
#                     help='resume from checkpoint')
# args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='/content/gdrive/My Drive/Colab Notebooks/data', train=True, download=True, transform=transform_train)
trainset = DatasetWithIndex(trainset)
print(len(trainset))


trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(
    root='/content/gdrive/My Drive/Colab Notebooks/data', train=False, download=True, transform=transform_test)
testset = DatasetWithIndex(testset)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
50000
Files already downloaded and verified


In [10]:
# Model
print('==> Building model..')
net = vgg.VGG('VGG16')
# net = ResNet18()
# net = PreActResNet18()
# net = GoogLeNet()
# net = DenseNet121()
# net = ResNeXt29_2x64d()
# net = MobileNet()
# net = MobileNetV2()
# net = DPN92()
# net = ShuffleNetG2()
# net = SENet18()
# net = ShuffleNetV2(1)
# net = EfficientNetB0()
# net = RegNetX_200MF()
# net = SimpleDLA()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

# if args.resume:
#     # Load checkpoint.
#     print('==> Resuming from checkpoint..')
#     assert os.path.isdir('checkpoint_all'), 'Error: no checkpoint directory found!'
#     checkpoint = torch.load('./checkpoint_all/ckpt_all.pth')
#     net.load_state_dict(checkpoint['net'])
#     best_acc = checkpoint['acc']
#     start_epoch = checkpoint['epoch']

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

==> Building model..


In [11]:
# Training
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets, sample_ids) in enumerate(trainloader):
        inputs, targets, sample_ids = inputs.to(device), targets.to(device), sample_ids.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # Update AUM
        aum_calculator.update(outputs, targets, sample_ids.tolist())

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        utils.progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                     % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [12]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets, sample_ids) in enumerate(testloader):
            inputs, targets, sample_ids = inputs.to(device), targets.to(device), sample_ids.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            utils.progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
            
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('/content/gdrive/My Drive/Colab Notebooks/model_save'):
            os.mkdir('/content/gdrive/My Drive/Colab Notebooks/model_save')
        torch.save(state, '/content/gdrive/My Drive/Colab Notebooks/model_save/ckpt_all.pth')
        best_acc = acc

In [13]:
# run
for epoch in range(start_epoch, start_epoch+100):
    train(epoch)
    # Finalize aum calculator
    aum_calculator.finalize()
    test(epoch)
    scheduler.step()


Epoch: 0
 [================================================================>]  Step: 411ms | Tot: 35s306ms | Loss: 1.472 | Acc: 46.510% (23255/50000)  391/391 
 [================================================================>]  Step: 46ms | Tot: 4s153ms | Loss: 1.138 | Acc: 59.720% (5972/10000)  100/100 
Saving..

Epoch: 1
 [================================================================>]  Step: 67ms | Tot: 35s492ms | Loss: 0.916 | Acc: 67.854% (33927/50000)  391/391 
 [================================================================>]  Step: 41ms | Tot: 4s232ms | Loss: 0.866 | Acc: 70.410% (7041/10000)  100/100 
Saving..

Epoch: 2
 [================================================================>]  Step: 61ms | Tot: 35s508ms | Loss: 0.719 | Acc: 75.154% (37577/50000)  391/391 
 [================================================================>]  Step: 41ms | Tot: 4s174ms | Loss: 0.782 | Acc: 74.260% (7426/10000)  100/100 
Saving..

Epoch: 3
 [==============================